In [1]:
import copy
import io
import numpy as np
import os
import pandas as pd
import pkgutil
import re
import time
import winsound

main_curr_dir = os.getcwd()
if os.path.basename(os.getcwd()) == 'scripts':
    os.chdir("../../")

import fermatrica as fm
import fermatrica_rep as fmr

import code_py.adhoc.model
import code_py.adhoc.reporting

metrics_max = 0
metrics_min = np.inf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Set environment

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# script settings

product_price = 1  # set current product price
conversion_rate = 1  # change only if some kind of convertion is expected

if_opt = True  # optimize / run w/o optimizing
cur_algo = 'local'  # 'global' / 'local'
if_visual = True  # switch on/off visuals
if_curves = True  # separate cause of media prices
if_save = False  # save model and for further analysis

## How to set model

1. Create or copy project with following structure:
    ```-[root]
        -code_py
            -adhoc
                -__init__.py
                -model.py [optional]
            -model_data
                -model_def.xlsx
            -scripts
                -model.ipynb
                
2. File "model_def.xlsx"
    1. Setup (model type, Y var, (product) price var, target brand, series frequency, conversion): sheet "setup"
    2. RHS (linear model specs): sheet "RHS"
    3. Transformations: sheet "params"
    4. Scoring (mostly for optimising): sheet "scoring"
    5. LHS (optional, Y transformations): sheet "LHS"
3. Data file
    1. Mandatory fields:
        1. "superbrand" (string): brand name
        2. "bs_key" (int): SKU / entity id
        3. "date" (datetime): date
        4. "listed" (int):
            - 1: before train period
            - 2: train period
            - 3: test period
            - 4: future
        5. any kind of target variable (as set in "model_def.xlsx")
        6. any kind of produce price variable (as set in "model_def.xlsx")

## Load data and current model definition

In [ ]:
dt_p = pd.read_excel('data/data_processed/data_for_sample_p00_w.xlsx', parse_dates=['date'])

# sync with FERMATRICA data structure

# 'superbrand' as brand name and 'bs_key' as SKU / entity id
dt_p['superbrand'] = 'brand_x'
dt_p['bs_key'] = 1

# 'date'
dt_p.rename({'Unnamed: 0': 'date'}, axis=1, inplace=True)

# 'Intercept' is not required
if 'Intercept' in dt_p.columns:
    del dt_p['Intercept']  # much faster than .drop()

# some kind of product price is required; set mock
dt_p['price'] = 1.0

# split dataset into pretrain, train, test, future
dt_p['listed'] = 1
dt_p.loc[(dt_p['date'] > '2013-06-01') & (dt_p['date'] <= '2017-07-31'), 'listed'] = 2
dt_p.loc[(dt_p['date'] > '2017-07-31') & (dt_p['date'] <= '2017-10-30'), 'listed'] = 3
dt_p.loc[(dt_p['date'] > '2017-10-30'), 'listed'] = 4
#

pth = os.path.join("code_py", "model_data", "model_def.xlsx")

model = fm.Model(path=pth
                    , adhoc_code=[code_py.adhoc.model]
                    , ds=dt_p)


start_time = time.time()

model = fm.transform(ds=dt_p
                        , model=model
                        , set_start=True
                        , if_by_ref=True)


print("time elapsed: " + str((time.time() - start_time)))

In [ ]:
dt_p.shape

## Optimising (optional)

In [ ]:
if if_opt:
    model_back = copy.deepcopy(model)

    if cur_algo == 'local':
        model = fm.optimize_local_cobyla(dt_p, model
                                                , revert_score_sign=True
                                                , verbose=True
                                                , epochs=1
                                                , iters_epoch=500
                                                , error_score=1e+12
                                                , ftol_abs=.00001)
        
    elif cur_algo == 'global':
        model = fm.optimize_global_ga(dt_p, model
                                            , revert_score_sign=False
                                            , epochs=1
                                            , verbose=True
                                            , iters_epoch=10
                                            , pop_size=20
                                            , pmutation=.1
                                            , max_no_improvement=10
                                            , error_score=-1e+12
                                            , cores=10
                                            , save_epoch=os.path.join(os.getcwd(), 'code_py', 'model_data'))

    model = copy.deepcopy(model)

    winsound.Beep(400, 1000)

## Get results

In [6]:
model = fm.transform(ds=dt_p
                      , model=model
                      , set_start=False
                      , if_by_ref=True)

pred, pred_raw, model = fm.fit_predict(dt_p, model, if_fit=True, return_full=True)
pred = fm.fit_predict(dt_p, model, if_fit=False, return_full=False)

dt_pred = fm.predict_ext(model, dt_p)


rsrc = pkgutil.get_data("fermatrica_rep", "/res/dict/vis_dict.xlsx")
xlsxio = io.BytesIO(rsrc)
vis_dict = pd.read_excel(xlsxio)

model_rep = fmr.ModelRep(dt_p, vis_dict)

In [ ]:
dt_p.shape

## Statistic and metrics

### Consolidated statistics

In [ ]:
print('\nScoring\n')

print('Full RMSE : ' + str(round(fm.metrics.rmse(dt_p.loc[dt_p['listed'].isin([2, 3]), model.conf.Y_var], pred[dt_p['listed'].isin([2, 3])]), 2)))
print('Full Combined scoring : ' + str(
    round(fm.scoring(dt_p.loc[dt_p['listed'].isin([2, 3]), :], pred[dt_p['listed'].isin([2, 3])], model), 5)))

print('\nFit\n')

print('R^2 train: ' + str(round(fm.metrics.r_squared(dt_p.loc[dt_p['listed'].isin([2]), model.conf.Y_var], pred[dt_p['listed'].isin([2])]), 4)))
print('MAPE train: ' + str(round(fm.metrics.mapef(dt_p.loc[dt_p['listed'].isin([2]), model.conf.Y_var], pred[dt_p['listed'].isin([2])]), 4)))
print('MAPE test: ' + str(round(fm.metrics.mapef(dt_p.loc[dt_p['listed'].isin([3]), model.conf.Y_var], pred[dt_p['listed'].isin([3])]), 4)))

print('\nTests\n')
tests_table = fmr.stats.tests_table_ols(model, dt_pred)

print(tests_table)

### Regression coefficients

In [ ]:
print('\nMain model\n')
print(model.obj.models['main'].summary())

if 'model_conf_back' in locals() or 'model_conf_back' in globals():

    tmp = pd.merge(model.conf.params[['variable', 'fun', 'arg', 'value']]
                    , model.conf_back.params[['variable', 'fun', 'arg', 'value']], on=['variable', 'fun', 'arg'])
    tmp = tmp[(tmp['value_x'] != tmp['value_y']) & (tmp['value_x'].notna())]

    tmp['diff'] = 0
    mask = (tmp['value_y'] != 0) & (pd.to_numeric(tmp['value_y'], errors='coerce').notna())
    tmp.loc[mask, 'diff'] = tmp.loc[mask, 'value_x'] / tmp.loc[mask, 'value_y'] - 1

    print('Changed params\n')
    tmp

### VIF (Multicollinearity)

In [ ]:
vif_tbl = fmr.stats.vif_table(model.obj.models['main'])
vif_tbl

## Observed and predicted

In [ ]:
if if_visual:
    
    fig = fmr.fit_main_plot_vol(model, dt_pred, model_rep, period='week', show_future=False)
    fig.show()

    fig = fmr.fit_main_plot_vol(model, dt_pred, model_rep, period='month', show_future=False)
    fig.show()

## Decomposition and Waterfall

In [ ]:
split_m_m = None

if if_visual:

    split_m_m = fmr.extract_effect(model, dt_p, model_rep)
    split_m_m = split_m_m.sort_values('date')

    fig = fmr.decompose_main_plot(split_m_m=split_m_m, brands=None, model_rep=model_rep
                                                        , period='m', show_future=False, contour_line=True)
    fig.show()

    fig = fmr.waterfall_plot(split_m_m=split_m_m, brands=None, model_rep=model_rep
                                                    , date_start='2013-01-01', date_end='2017-10-31')
    fig.show()


### Simple curves

In [ ]:
# set media prices in "model_def.xlsx", sheet "trans_path_df", before running

curves_data = None

if if_curves:
    
    curves_data = fmr.curves_simple_data(model=model
                                        , ds=dt_p
                                        , model_rep=model_rep
                                        , budget_lim=50  # in millions
                                        , budget_step=.01  # in millions
                                        , if_precise=False
                                        )

    fig = fmr.curves_simple_plot(ds=curves_data
                                , model_rep=model_rep
                                , price=product_price
                                , conv=conversion_rate
                                )

    fig['Incremental Volume'].show()
    fig['Incremental Value'].show()

## Save or export results

In [14]:
pth = ''

if if_save:

    pth = model.save(dt_p, path=os.path.join(os.getcwd(), 'code_py', 'model_data'))
